In [1]:
import numpy as np
import os
import tensorflow as tf
from sklearn.metrics import balanced_accuracy_score

2023-01-21 13:30:03.958616: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-21 13:30:04.496085: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-21 13:30:06.437411: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib

In [2]:
image_count_ = 901 
batch_size_ = 8

img_height_ = 256
img_width_ = 256

AUTOTUNE = tf.data.AUTOTUNE

In [3]:
import warnings

warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')

os.environ["TF_GPU_ALLOCATOR"] = 'cuda_malloc_async'

In [4]:
tf.config.run_functions_eagerly(True)

In [5]:
# data_dir_2 = "drive/MyDrive/projects/mit_indoor_scence_recognition/datasets/Images/*/*"
dataset_address = "/home/ubuntu/projects/mit_indoor_scene/data/data_test"

In [6]:
training_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_address,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size= batch_size_,
    image_size=(img_height_, img_width_),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training',
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 901 files belonging to 5 classes.
Using 721 files for training.


2023-01-14 17:08:40.674499: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 17:08:40.683727: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 17:08:40.685844: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 17:08:40.688507: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operati

In [7]:
validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_address,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=batch_size_,
    image_size=(img_height_, img_width_),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation',
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 901 files belonging to 5 classes.
Using 180 files for validation.


In [8]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = training_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = validation_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [9]:
def b_acc(y_true, y_pred):
      
    Y_true = []
    Y_pred = []

    for i in range(y_true.shape[0]):
        i_ = np.argmax(y_true[i,:])
        j_ = np.argmax(y_pred[i,:])
        Y_true.append(i_+1)
        Y_pred.append(j_+1)
    
    
    return balanced_accuracy_score(Y_true, Y_pred)

In [10]:
# num_classes = 5


aug_model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(256, 256, 3)),
  tf.keras.layers.Rescaling(1./255),

  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),
  # tf.keras.layers.RandomHeight(0.2, interpolation='bilinear'),
  tf.keras.layers.RandomTranslation(0.2, 0.2, fill_mode='reflect', interpolation='bilinear', fill_value=0.0),
  # tf.keras.layers.RandomWidth(0.2, interpolation='bilinear'),
  tf.keras.layers.RandomZoom(0.2, 0.2, fill_mode='reflect', interpolation='bilinear')  
  # tf.keras.layers.Flatten()
])

In [11]:
input_layer = tf.keras.layers.Input(shape=(256, 256, 3))
output_layer = tf.keras.layers.Dense(1)


def get_model(input_layer_):

  augmented_input = aug_model(input_layer_)
#   conv_1 = tf.keras.layers.Conv2D(512, 3, padding='same', activation='relu', input_shape=(256, 256, 3))(augmented_input)
#   conv_1 = tf.keras.layers.MaxPooling2D()(conv_1)

  conv_2 = tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu', input_shape=(256, 256, 3))(augmented_input)
  conv_2 = tf.keras.layers.MaxPooling2D()(conv_2)

  conv_3 = tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu', input_shape=(256, 256, 3))(conv_2)
  conv_3 = tf.keras.layers.MaxPooling2D()(conv_3)

  conv_4 = tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu', input_shape=(256, 256, 3))(conv_3)
  conv_4 = tf.keras.layers.MaxPooling2D()(conv_4)

  conv_5 = tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu')(conv_4)
  conv_5 = tf.keras.layers.MaxPooling2D()(conv_5)

  conv_5 = tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu')(conv_5)
  conv_5 = tf.keras.layers.MaxPooling2D()(conv_5)

  flatten = tf.keras.layers.Flatten()(conv_5)
#   dense_1 = tf.keras.layers.Dense(256, activation='relu')(flatten)
  dense_2 = tf.keras.layers.Dense(128, activation='relu')(flatten)
  dense_3 = tf.keras.layers.Dense(32, activation='relu')(dense_2)
  dense_4 = tf.keras.layers.Dense(1, activation='softmax')(dense_3)

  return dense_4


last_layers = list()

classes_num = 5

for _ in range(classes_num):
  last_layer = get_model(input_layer)
  last_layers.append(last_layer)


output_layer = tf.keras.layers.Concatenate()([ layer_ for layer_ in last_layers ])

final_model = tf.keras.Model(input_layer, output_layer, name="encoder")

In [12]:
final_model.compile(
  optimizer='adam',
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
  metrics=[b_acc])

In [ ]:
history = final_model.fit(
              train_ds,
              validation_data=val_ds,
              epochs=10,
              verbose=1
            )

Epoch 1/10


2023-01-14 17:09:21.384094: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8400


91/91 [==============================] - 1093s 12s/step - loss: 1.6094 - b_acc: 0.1740 - val_loss: 1.6094 - val_b_acc: 0.2036
Epoch 2/10
91/91 [==============================] - 1095s 12s/step - loss: 1.6094 - b_acc: 0.1740 - val_loss: 1.6094 - val_b_acc: 0.2036
Epoch 3/10
65/91 [====================>.........] - ETA: 5:12 - loss: 1.6094 - b_acc: 0.1923